In [1]:
import os 
import time 
import random
import numpy as np 

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2

from tensorflow.keras.callbacks import TensorBoard

from data_prepare.save_data import save
from data_prepare.data_splitter import get_train_and_test_data as gttd
from service.service_data import ServiceData as sd

In [2]:
level = 'SnowballSurvival'
save(level, delete=True, debug=True)
os.chdir('D:/Documents/Python/fallguy_bot/DATA/LEVELS/'+ level)

Reading: D:/Documents/Python/fallguy_bot/DATA/LEVELS/SnowballSurvival
Run # 1
Dead categories= 99
Fine categories= 1479
Raw  categories= 1578
raw  img list= 1578
Fine img list= 1479
Run # 2
Dead categories= 115
Fine categories= 1290
Raw  categories= 1405
raw  img list= 1405
Fine img list= 1290
i found  2769  Data
and 2769 numbers of Categories
Saving to Numpy in D:\Documents\Python\fallguy_bot\DATA\LEVELS\SnowballSurvival
Done! Saving


In [3]:
X = np.load(open('X.npy','rb'))
y = np.load(open('y.npy','rb'))


In [4]:
X_train, y_train, X_test, y_test = gttd(X,y)

press_number in key = {'W_nums': 49, 'A_nums': 102, 'S_nums': 186, 'D_nums': 61, 'SPACE_nums': 0, 'E_nums': 0}
max press = 186
In Key = A_nums, press len list = 186
In Key = S_nums, press len list = 186
In Key = D_nums, press len list = 186
In Key = SPACE_nums, press len list = 0
In Key = E_nums, press len list = 0
557
creating X_test
creating y test
creating X_train
creating y_train
try to return


In [5]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(502, 120, 200, 1)
(502, 6)
(55, 120, 200, 1)
(55, 6)


In [ ]:
del X, y
X_train = X_train/255.0
X_test = X_test/255.0

y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')




In [7]:
what_label = y_train
new_dict_key = dict(list(sd.keys_categories.items())[1:])
for key_pos, key_name in enumerate(new_dict_key):
    press_number = 0 
    for label in what_label:
        for index, num in enumerate(label):
            if num == 1 and index==key_pos:
                press_number += 1

    print(f'In {len(what_label)} categories. You pressed key with position {key_name}:')
    print(f'{press_number} times')
    print('its have {:.2f} percent'.format(press_number*100/len(what_label)))
    print(f'________________________________________________________________________')

print(y_train[10][1:])
print(y_train[10])

In 502 categories. You pressed key with position A:
210 times
its have 41.83 percent
________________________________________________________________________
In 502 categories. You pressed key with position S:
269 times
its have 53.59 percent
________________________________________________________________________
In 502 categories. You pressed key with position D:
178 times
its have 35.46 percent
________________________________________________________________________
In 502 categories. You pressed key with position SPACE:
0 times
its have 0.00 percent
________________________________________________________________________
In 502 categories. You pressed key with position E:
0 times
its have 0.00 percent
________________________________________________________________________
In 502 categories. You pressed key with position SHIFT:
0 times
its have 0.00 percent
________________________________________________________________________
In 502 categories. You pressed key with position NOTH

In [9]:
X_train.shape[1:]
y_train.shape[1:]


(6,)

In [ ]:
#My fucked up model 
# model = Sequential()

# model.add(Conv2D((64), (3,3), input_shape = X_train.shape[1:]))
# model.add(Activation("relu"))
# model.add(MaxPooling2D(pool_size=(2,2)))

# model.add(Conv2D((32), (2,2)))
# model.add(Activation("relu"))
# model.add(MaxPooling2D(pool_size=(2,2)))


# model.add(Flatten())
# model.add(Dense(8))
# model.add(Activation('relu'))


# model.add(Dense(7))
# model.add(Activation('sigmoid'))

# model.compile(
#     loss="binary_crossentropy",
#     optimizer='adam',
#     metrics=['accuracy'],
# )
# model.summary()

model = Sequential()

# 2 sets of CRP (Convolution, RELU, Pooling)
model.add(Conv2D(20, (5, 5), padding="same",
	input_shape=X_train.shape[1:], kernel_regularizer=l2(0.)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(50, (5, 5), padding="same",
	kernel_regularizer=l2(0.)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Fully connected layers (w/ RELU)
model.add(Flatten())
model.add(Dense(500, kernel_regularizer=l2(0.)))
model.add(Activation("relu"))

# Softmax (for classification)
model.add(Dense(len(y_train[0]), kernel_regularizer=l2(0.)))
model.add(Activation("softmax"))

model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

In [ ]:
NAME = 'CNN-{}'.format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

In [ ]:
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_test, y_test), callbacks=[tensorboard])
model.save(f'cnn_{level}.model')